In [34]:
import sqlite3
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Step 1: Connect to SQLite Database and Extract Data
conn = sqlite3.connect('chinook.db')

# SQL query to get all tracks purchased in each invoice
query = """
SELECT 
    invoices.InvoiceId,
    tracks.Name
FROM 
    invoice_items
INNER JOIN 
    invoices ON invoice_items.InvoiceId = invoices.InvoiceId
INNER JOIN 
    tracks ON invoice_items.TrackId = tracks.TrackId;
"""

# Execute the query and load the result into a pandas DataFrame
df = pd.read_sql_query(query, conn)
print(df)

# Close the database connection
conn.close()


      InvoiceId                           Name
0            98            Experiment In Terra
1            98              Take the Celestra
2           121              Shout It Out Loud
3           121               Calling Dr. Love
4           121                       Strutter
...         ...                            ...
2235        284           Esporrei Na Manivela
2236        284  No Fundo Do Quintal Da Escola
2237        284                 Que Luz É Essa
2238        284          The Power Of Equality
2239        284   Mellowship Slinky In B Major

[2240 rows x 2 columns]


In [35]:
# Step 2: Group the Data by InvoiceId and Create a List of Transactions
transactions = df.groupby('InvoiceId')['Name'].apply(list).tolist()


In [36]:
# Step 3: Convert the Transaction Data to a Binary Matrix
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
binary_df = pd.DataFrame(te_ary, columns=te.columns_)

print(binary_df.head())

     "?"  #9 Dream  'Round Midnight  (Anesthesia) Pulling Teeth  \
0  False     False            False                       False   
1  False     False            False                       False   
2  False     False            False                       False   
3  False     False            False                       False   
4  False     False            False                       False   

   (White Man) In Hammersmith Palais  (Wish I Could) Hideaway  ...And Found  \
0                              False                    False         False   
1                              False                    False         False   
2                              False                    False         False   
3                              False                    False         False   
4                              False                    False         False   

   ...And Justice For All  01 - Prowler  04 - Running Free  ...  Zé Trindade  \
0                   False         False   

In [ ]:
# Step 4: Apply the Apriori Algorithm to Find Frequent Itemsets
# Set a minimum support threshold (e.g., 0.001 means an itemset appears in 0.5% of transactions)
frequent_itemsets = apriori(binary_df, min_support=0.005, use_colnames=True)


In [ ]:

# Step 5: Generate Association Rules from the Frequent Itemsets
# Set a minimum confidence threshold (e.g., 0.01 means a rule is considered if it holds 1% of the time)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)


In [ ]:

# Step 6: Export the Results to CSV Files
# Export the binary transaction dataset
binary_df.to_csv('binary_transactions.csv', index=False)

# Export the frequent itemsets
frequent_itemsets.to_csv('frequent_itemsets.csv', index=False)

# Export the association rules
rules.to_csv('association_rules.csv', index=False)


In [ ]:

# Step 7: Display Key Insights
print("Frequent Itemsets:")
print(frequent_itemsets)

print("\nAssociation Rules:")
print(rules)

import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the rules
plt.figure(figsize=(10, 6))
sns.scatterplot(x="support", y="confidence", size="lift", hue="lift", data=rules)
plt.title('Support vs Confidence')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.show()



In [ ]:
binary_df.to_csv('binary_data1.csv', index=False)
frequent_itemsets.to_csv('frequent_itemsets1.csv', index=False)
rules.to_csv('association_rules1.csv', index=False)